In [1]:
import numpy as np
import pandas as pd

In [4]:
df=pd.read_csv("../TextFiles/smsspamcollection.tsv",sep='\t')

In [5]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [6]:
X=df['message']
y=df['label']

In [21]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=41)

In [22]:
X_train.shape

(3733,)

## Scikit-learn's CountVectorizer


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()


In [30]:
count_vect_obj=count_vect.fit(X_train)
X_train_count_vector=count_vect_obj.transform(X_train)
X_test_count_vector=count_vect_obj.transform(X_test)
X_train_count_vector,X_test_count_vector


(<3733x7078 sparse matrix of type '<class 'numpy.int64'>'
 	with 49841 stored elements in Compressed Sparse Row format>,
 <1839x7078 sparse matrix of type '<class 'numpy.int64'>'
 	with 22414 stored elements in Compressed Sparse Row format>)

## traning with countvectorization

In [31]:
from sklearn.svm import LinearSVC
svm=LinearSVC()
svm.fit(X_train_count_vector,y_train)


LinearSVC()

In [35]:
predictions=svm.predict(X_test_count_vector)

In [36]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1570    2]
 [  31  236]]


In [52]:
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1572
        spam       0.99      0.88      0.93       267

    accuracy                           0.98      1839
   macro avg       0.99      0.94      0.96      1839
weighted avg       0.98      0.98      0.98      1839



## training with tfidf

In [53]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf=tfidf_transformer.fit_transform(X_train_count_vector)
X_test_tfidf=tfidf_transformer.fit_transform(X_test_count_vector)
# X_train_tfidf,X_test_tfidf

In [54]:
from sklearn.svm import LinearSVC
svm=LinearSVC()
svm.fit(X_train_tfidf,y_train)
predictions=svm.predict(X_test_tfidf)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))

[[1570    2]
 [  22  245]]
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1572
        spam       0.99      0.92      0.95       267

    accuracy                           0.99      1839
   macro avg       0.99      0.96      0.97      1839
weighted avg       0.99      0.99      0.99      1839



## pipeline

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [57]:
predictions = text_clf.predict(X_test)

In [58]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))

[[1570    2]
 [  23  244]]
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1572
        spam       0.99      0.91      0.95       267

    accuracy                           0.99      1839
   macro avg       0.99      0.96      0.97      1839
weighted avg       0.99      0.99      0.99      1839

